In [1]:
import os

KAGGLE =  os.getenv("KAGGLE_URL_BASE") is not None
COLAB = os.getenv("COLAB_GPU") is not None
TPU = os.getenv("XRT_TPU_CONFIG") is not None
LOCAL = not KAGGLE and not COLAB

if not LOCAL:
    !git clone https://github.com/nclibz/MRKnee/

if COLAB:
    os.chdir('/content/MRKnee/')
    !git checkout v3
    from google.colab import drive
    drive.mount('/content/drive')
    DATADIR = "/content/drive/MyDrive/MRKnee/data"
    if TPU:
        !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

if KAGGLE:
    os.chdir('/kaggle/working/MRKnee/')
    !git checkout v3
    DATADIR = "/kaggle/input/mrknee/MRNet"
    
    if TPU:
        !pip install torchtext==0.9
        !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
        !python pytorch-xla-env-setup.py --version 1.8

if not LOCAL:
    !pip install --quiet "pytorch-lightning>=1.4.9" "torchmetrics>=0.5" "timm" "neptune-client" "optuna" "PyMySql" "torch-tb-profiler"
    !pip install albumentations --upgrade --quiet
    BACKBONE = "tf_efficientnetv2_s_in21k"

if LOCAL:
    DATADIR = "data"
    BACKBONE = 'mobilenetv3_large_100_miil_in21k'
    %load_ext autoreload
    %autoreload 2

In [2]:
DIAGNOSIS = "acl"
PLANE = "sagittal"

In [3]:
from src.study import Study
from src.model import MRKnee
from src.data import MRKneeDataModule
from src.augmentations import Augmentations
from src.callbacks import Callbacks
from src.cfg import Cfg
import pytorch_lightning as pl

pl.seed_everything(123)

Global seed set to 123


123

In [8]:

def objective(trial, diagnosis=DIAGNOSIS, plane=PLANE, backbone=BACKBONE, datadir=DATADIR, profile = False):

    model = MRKnee(
        backbone=backbone,
        drop_rate=trial.suggest_int("drop_rate", 30, 80) / 100,
        final_drop=trial.suggest_int("final_drop", 0, 15) / 100,
        learning_rate=trial.suggest_loguniform('lr', 1e-6, 1e-3),
        log_auc=True,
        log_ind_loss=False,
        adam_wd=trial.suggest_loguniform('adam_wd', 0.001, 0.1),
        max_epochs=20,
        precision=32,
    )

    augs = Augmentations(
        model,
        shift_limit=trial.suggest_int("shift_limit", 0, 15) / 100,
        scale_limit=trial.suggest_int("scale_limit", 0, 15) / 100,
        rotate_limit=trial.suggest_int("rotate_limit", 0, 15) / 100,
        reverse_p=0.5,
        indp_normalz=True,
    )

    dm = MRKneeDataModule(
        datadir=datadir,
        diagnosis=diagnosis,
        plane=plane,
        transforms=augs,
        clean=True,
        num_workers=2,
        pin_memory=True,
        trim_train=True,
    )

    configs = Cfg(model = model, dm = dm, augs = augs)
    cfg = configs.get_cfg()
    
    if trial is not None:
        callbacks = Callbacks(cfg, trial, neptune_name="v3")
        neptune_logger = callbacks.get_neptune_logger()
        list_of_cbs = callbacks.get_callbacks()
        fast_dev_run = False
    else:
        neptune_logger = False
        list_of_cbs = None
        fast_dev_run = 50

    profiler = pl.profiler.PyTorchProfiler(dirpath = "src/logs", filename = "profiler") if profile else False
        


    trainer = pl.Trainer(
        gpus=1,
        precision=cfg["precision"],
        max_epochs=cfg["max_epochs"],
        logger=neptune_logger,
        log_every_n_steps=100,
        num_sanity_val_steps=0,
        callbacks=list_of_cbs,
        progress_bar_refresh_rate=20,
        deterministic=True,
        profiler = profiler, 
        fast_dev_run = fast_dev_run,
    )

    trainer.fit(model, dm)

    ## UPLOAD BEST CHECKPOINTS TO LOG
    if trial is not None:
        callbacks.upload_best_checkpoints()

    return callbacks.model_checkpoint.best_model_score.item()


In [9]:
# For testing
#objective(trial = None)

study = Study(diagnosis = DIAGNOSIS,
              plane = PLANE,
              backbone=BACKBONE,
              min_epochs=10)

In [10]:

study.optimize(objective, n_trials=40, timeout=8 * 60 * 60)

/home/nicolai/miniconda3/envs/dl/lib/python3.9/site-packages/optuna/samplers/_tpe/sampler.py:263: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2021-10-08 13:44:37,168] Using an existing study with name 'acl_sagittal_tf_efficientnetv2_s_in21k' instead of creating a new one.
NeptuneLogger will work in online mode
Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type         | Params
--------------------------------------------
0 | backbone   | EfficientNet | 20.2 M
1 | final_drop | Dropout      | 0     
2 | clf        | Linear       | 1.3 K 
--------------------------------------------
20.2 M    Trainable params
0         Non-trainable params
20.2 M    Total params
80.713    Total estimated model params size (MB)
/home/nicolai/miniconda3/envs/dl/lib/pyth

Training: -1it [00:00, ?it/s]https://ui.neptune.ai/nclibz/tester/e/TES-15
Epoch 0:   0%|          | 0/1241 [00:00<00:01, 1042.32it/s]  

[W 2021-10-08 13:45:27,132] Trial 4 failed because of the following error: RuntimeError('CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 3.82 GiB total capacity; 2.40 GiB already allocated; 54.38 MiB free; 2.47 GiB reserved in total by PyTorch)')
Traceback (most recent call last):
  File "/home/nicolai/miniconda3/envs/dl/lib/python3.9/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_41922/3728728504.py", line 67, in objective
    trainer.fit(model, dm)
  File "/home/nicolai/miniconda3/envs/dl/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 552, in fit
    self._run(model)
  File "/home/nicolai/miniconda3/envs/dl/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 922, in _run
    self._dispatch()
  File "/home/nicolai/miniconda3/envs/dl/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 990, in _dispatch
    self.accelerator.start_training(s

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 3.82 GiB total capacity; 2.40 GiB already allocated; 54.38 MiB free; 2.47 GiB reserved in total by PyTorch)